In [2]:
import pandas as pd
import geopandas as gpd
import os
import overpass

Download the [OS Open Names dataset](https://osdatahub.os.uk/data/downloads/open/OpenNames) and extract the zip into this directory, it should produce a directory called `opname_csv_gb`.

In [15]:
with open("opname_csv_gb/Doc/OS_Open_Names_Header.csv") as f:
    header = f.readline().split(',')

results = pd.DataFrame()
with os.scandir("opname_csv_gb/Data") as it:
    for entry in it:
        if entry.is_file() and entry.name.endswith(".csv"):
            df = pd.read_csv(entry.path, header=None, names=header)
            subset = df[df['TYPE'] == "populatedPlace"]
            results = pd.concat([results, subset[['NAME1', 'NAME2', 'GEOMETRY_X', 'GEOMETRY_Y']]])

/tmp/ipykernel_22369/3634243814.py:8: DtypeWarning: Columns (4,16,17,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_22369/3634243814.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_22369/3634243814.py:8: DtypeWarning: Columns (4,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_22369/3634243814.py:8: DtypeWarning: Columns (4,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_22369/3634243814.py:8: DtypeWarning: Columns (4,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None

In [16]:
gdf = gpd.GeoDataFrame(results, geometry=gpd.points_from_xy(results['GEOMETRY_X'], results['GEOMETRY_Y'], crs=27700))
gdf = gdf.to_crs(4326)
gdf['lat'] = round(gdf.geometry.y, 3)
gdf['lon'] = round(gdf.geometry.x, 3)
gb_names = gdf[['NAME1', 'NAME2', 'lat', 'lon']].set_index(['lat', 'lon']).stack().reset_index(name='name').drop(columns='level_2').drop_duplicates()

Download the [OSNI Gazetteer](https://admin.opendatani.gov.uk/dataset/osni-open-data-gazetteer-place-names).

In [64]:
df = pd.read_csv('https://admin.opendatani.gov.uk/dataset/d27903f1-15e6-4c07-8564-ddc655e9c549/resource/411c6ea5-c079-43aa-bd62-438423d070cc/download/osni_open_data_-_gazetteer_-_place_names.csv')
igdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['X'], df['Y'], crs=29903))
igdf = igdf.to_crs(4326)
igdf['lat'] = round(igdf.geometry.y, 3)
igdf['lon'] = round(igdf.geometry.x, 3)
ni_names = igdf[['PLACENAME', 'lat', 'lon']].rename(columns={'PLACENAME': 'name'})
ni_names['name'] = ni_names['name'].str.title()

Download the [OpenStreetMap Names dataset](https://osmnames.org/), apply a bounding box and extract places. Then look up the places from Overpass to get Irish and English names.

In [3]:
planet = pd.read_table('https://github.com/OSMNames/OSMNames/releases/download/v2.0.4/planet-latest_geonames.tsv.gz', iterator=True, chunksize=100000, compression='gzip')
ireland = pd.concat([chunk[(chunk['lon'] > -10.56) & (chunk['lon'] < -5.34) & (chunk['lat'] > 51.39) & (chunk['lat'] < 55.43)] for chunk in planet])

/tmp/ipykernel_54597/2108726369.py:2: DtypeWarning: Columns (10,23) have mixed types. Specify dtype option on import or set low_memory=False.
  ireland = pd.concat([chunk[(chunk['lon'] > -10.56) & (chunk['lon'] < -5.34) & (chunk['lat'] > 51.39) & (chunk['lat'] < 55.43)] for chunk in planet])
/tmp/ipykernel_54597/2108726369.py:2: DtypeWarning: Columns (10,23) have mixed types. Specify dtype option on import or set low_memory=False.
  ireland = pd.concat([chunk[(chunk['lon'] > -10.56) & (chunk['lon'] < -5.34) & (chunk['lat'] > 51.39) & (chunk['lat'] < 55.43)] for chunk in planet])
/tmp/ipykernel_54597/2108726369.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  ireland = pd.concat([chunk[(chunk['lon'] > -10.56) & (chunk['lon'] < -5.34) & (chunk['lat'] > 51.39) & (chunk['lat'] < 55.43)] for chunk in planet])
/tmp/ipykernel_54597/2108726369.py:2: DtypeWarning: Columns (10,23) have mixed types. Specify dtype option on import or set 

In [4]:
ireland_trimmed = ireland[(ireland['lon'] < -5.85) | (ireland['lat'] < 55.23)] # Cut off corner of Scotland
ireland_trimmed = ireland_trimmed[(ireland_trimmed['lon'] < -5.48) | (ireland_trimmed['lat'] > 51.91)] # Cut off corner of Wales
places = ireland_trimmed[(ireland_trimmed['class'] == 'place') & (ireland_trimmed['type'] != 'quarter') & (ireland_trimmed['type'] != 'state')]
places.set_index('osm_id', inplace=True)

In [5]:
api = overpass.API(timeout=600)

In [136]:
def get_names(api, df, osm_type):
    idstr = ','.join(df[df['osm_type'] == osm_type].index.astype(str).to_list())
    query = f"{osm_type}(id:{idstr});"
    elements = api.get(query, verbosity='tags', responseformat='json').get('elements')
    return {e['id']: {'ga': e.get('tags', {}).get('name:ga'), 'en': e.get('tags', {}).get('name:en'), 'name': e.get('tags', {}).get('name')} for e in elements}

relations = get_names(api, places, 'relation')


In [137]:
ways = get_names(api, places, 'way')

In [130]:
nodes = get_names(api, places, 'node')

In [138]:
names = pd.DataFrame.from_dict(ways | nodes | relations, orient='index')
minimal = places.merge(names, left_index=True, right_index=True, how='left').reset_index()[['name_x', 'ga', 'en', 'lat', 'lon']]
minimal.rename(columns={'name_x': 'name'}, inplace=True)

In [139]:
ireland_names = minimal.set_index(['lat', 'lon']).stack().reset_index(name='name').drop(columns='level_2').drop_duplicates()
ireland_names['lat'] = round(ireland_names.lat, 3)
ireland_names['lon'] = round(ireland_names.lon, 3)

In [142]:
places[places['country'] == 'United Kingdom']


,name,alternative_names,osm_type,class,type,lon,lat,place_rank,importance,street,...,country,country_code,display_name,west,south,east,north,wikidata,wikipedia,housenumbers
osm_id,,,,,,,,,,,,,,,,,,,,,
1418701024,Belfast,"벨파스트,ベルファスト,Béal Feirste,Beeal Feirshtey,Belfa...",node,place,city,-5.930099,54.596987,16.0,0.665796,NaN,...,United Kingdom,gb,"Belfast, Northern Ireland, United Kingdom",-5.930099,54.596987,-5.930099,54.596987,Q10686,en:Belfast,NaN
267762522,Londonderry/Derry,"Derae,Derry,Doire,Londonderis,Stroke City,ديري...",node,place,city,-7.316801,54.991942,16.0,0.567408,NaN,...,United Kingdom,gb,"Londonderry/Derry, Northern Ireland, United Ki...",-7.316801,54.991942,-7.316801,54.991942,Q163584,en:Derry,NaN
267763062,Armagh,"Ard Mhacha,Arma,Арма",node,place,city,-6.654043,54.348198,16.0,0.522952,NaN,...,United Kingdom,gb,"Armagh, Northern Ireland, United Kingdom",-6.654043,54.348198,-6.654043,54.348198,Q193452,en:Armagh,NaN
1460517034,Lisburn,"Lios na gCearrbhach,Lisbernas,Лисберн,Лісберн",node,place,city,-6.035314,54.512935,16.0,0.518118,NaN,...,United Kingdom,gb,"Lisburn, Northern Ireland, United Kingdom",-6.035314,54.512935,-6.035314,54.512935,Q210647,en:Lisburn,NaN
663428001,Newry,"Iubhar Cinn Trágha,Niūris,Ньюри,Ньюрі",node,place,city,-6.337506,54.177528,16.0,0.515036,NaN,...,United Kingdom,gb,"Newry, Northern Ireland, United Kingdom",-6.337506,54.177528,-6.337506,54.177528,Q269980,en:Newry,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242416311,Crannog,NaN,way,place,island,-6.798135,54.290398,17.0,0.325000,NaN,...,United Kingdom,gb,"Crannog, Northern Ireland, United Kingdom",-6.798374,54.290271,-6.797893,54.290577,NaN,NaN,NaN
3516250,Inishfendra,NaN,relation,place,island,-7.429117,54.158051,17.0,0.325000,NaN,...,United Kingdom,gb,Inishfendra,-7.443510,54.154158,-7.412868,54.161859,NaN,NaN,NaN
3512150,Boa Island,"Badhbha Island,Beau Island",relation,place,island,-7.830899,54.517350,17.0,0.324102,NaN,...,United Kingdom,gb,"Boa Island, Northern Ireland, United Kingdom",-7.897895,54.505315,-7.786451,54.529372,Q887472,en:Boa Island,NaN


In [145]:
output = pd.concat([gb_names, ireland_names])[['name','lat','lon']].to_json(orient='values')
with open("app/places.json", "w") as fd:
    fd.write(output)

In [86]:
def places_in_bb(api, latmin, lonmin, latmax, lonmax, type='node', placetypes=['city','town','village','hamlet','island','neighbourhood','suburb','locality']):
  return api.get(f"""
    {type}[place~"{'|'.join(placetypes)}"]({latmin}, {lonmin}, {latmax}, {lonmax});
  """, responseformat='json', verbosity='body')

p = places_in_bb(api, 51.39, -10.56, 55.43, -5.34)
op = pd.DataFrame.from_dict(p['elements'])

In [87]:
live = pd.concat([op.drop(['tags'], axis=1), op['tags'].apply(pd.Series)], axis=1)[['type','place','id','name','alt_name','name:ga','name:en','lat','lon']]
live.set_index('id', inplace=True)

In [88]:
pd.merge(live, places, left_index=True, right_index=True, how='inner')

,type_x,place,name_x,alt_name,name:ga,name:en,lat_x,lon_x,name_y,alternative_names,...,country,country_code,display_name,west,south,east,north,wikidata,wikipedia,housenumbers
8563682,node,suburb,Mayfield,NaN,Baile na mBocht,NaN,51.912439,-8.432801,Mayfield,Baile na mBocht,...,Ireland,ie,"Mayfield, Montenotte B, Ireland",-8.432802,51.912438,-8.432802,51.912438,NaN,NaN,NaN
24905097,node,hamlet,Garvary,NaN,NaN,NaN,54.358967,-7.577444,Garvary,NaN,...,Ireland,ie,"Garvary, Ireland",-7.577444,54.358967,-7.577444,54.358967,NaN,NaN,NaN
31078662,node,village,Rosslare Harbour,Ballygeary,NaN,NaN,52.249733,-6.340115,Rosslare Harbour,Рослэр-Харбор,...,Ireland,ie,"Rosslare Harbour, Ireland",-6.334291,52.253358,-6.334291,52.253358,NaN,NaN,NaN
31086076,node,town,Carrick-on-Shannon,Carrick on Shannon,Cora Droma Rúisc,Carrick-on-Shannon,53.944146,-8.095277,Carrick on Shannon,Каррик,...,Ireland,ie,"Carrick on Shannon, Ireland",-8.095270,53.944159,-8.095270,53.944159,NaN,NaN,NaN
31241248,node,village,Ballyleague,NaN,NaN,NaN,53.675321,-7.995412,Ballyleague,NaN,...,Ireland,ie,"Ballyleague, Cloontuskert Electoral Division, ...",-7.995412,53.675321,-7.995412,53.675321,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5219708484,node,hamlet,Barrack Bog,NaN,NaN,NaN,53.703609,-7.796207,Barrack Bog,NaN,...,Ireland,ie,"Barrack Bog, Longford Rural ED, Ireland",-7.796207,53.703609,-7.796207,53.703609,NaN,NaN,NaN
5226028170,node,village,Ballysadare,NaN,Baile Easa Dara,Ballysadare,54.209888,-8.509265,Ballysadare,Baile Easa Dara,...,Ireland,ie,"Ballysadare, Collooney ED, Ireland",-8.509265,54.209888,-8.509265,54.209888,NaN,NaN,NaN
5250620392,node,village,Daingean,NaN,An Daingean,Daingean,53.297505,-7.290978,Daingean,Philipstown,...,Ireland,ie,"Daingean, Daingean Electoral Division, Ireland",-7.292229,53.297345,-7.292229,53.297345,NaN,NaN,NaN
5286382091,node,town,Lurgan,NaN,An Lorgain,NaN,54.460824,-6.329249,Lurgan,An Lorgain,...,Ireland,ie,"Lurgan, Ireland",-6.329249,54.460824,-6.329249,54.460824,NaN,NaN,NaN


In [ ]:
live.to_csv('live.csv', index=False)

In [89]:
nigeo = gpd.GeoDataFrame(ni_names['name'], geometry=gpd.points_from_xy(ni_names['lon'], ni_names['lat'], crs=4326))
livegeo = gpd.GeoDataFrame(live['name'], geometry=gpd.points_from_xy(live['lon'], live['lat'], crs=4326))

In [90]:
merged = pd.merge(livegeo, nigeo, left_on='name', right_on='name', how='inner')
merged['distance'] = merged['geometry_x'].distance(merged['geometry_y'])
merged = merged[merged['distance'] < 0.2].sort_values('distance')

/tmp/ipykernel_54597/969134632.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged['distance'] = merged['geometry_x'].distance(merged['geometry_y'])


In [91]:
combined = ni_names.merge(merged, left_on='name', right_on='name', how='outer', indicator=True)
missing = combined[combined['_merge'] == 'left_only']
missing

,name,lat,lon,geometry_x,geometry_y,distance,_merge
0,"""Kells Connor""",54.808,-6.220,None,None,NaN,left_only
5,Agivey,55.045,-6.594,None,None,NaN,left_only
22,Baileysmill,54.459,-5.959,None,None,NaN,left_only
25,Ballybogy,55.130,-6.557,None,None,NaN,left_only
29,Ballygalley,54.898,-5.859,None,None,NaN,left_only
50,Belcoo,54.291,-6.765,None,None,NaN,left_only
69,Bovedy,54.945,-6.609,None,None,NaN,left_only
78,Cappagh,54.542,-6.925,None,None,NaN,left_only
104,Clogh,54.967,-6.289,None,None,NaN,left_only
105,Clogh Mills,55.000,-6.332,None,None,NaN,left_only


In [93]:
live[live['name'] == 'Annadorn']

,type,place,name,alt_name,name:ga,name:en,lat,lon
id,,,,,,,,
11153263341,node,locality,Annadorn,NaN,Eanach Dorn,NaN,54.331527,-5.800202


In [ ]:
live['lat'] = round(live.lat, 3)
live['lon'] = round(live.lon, 3)
live_names = live[['name','alt_name','name:ga','name:en','lat','lon']].set_index(['lat', 'lon']).stack().reset_index(name='name').drop(columns='level_2').drop_duplicates()
output = live_names[['name','lat','lon']].to_json(orient='values')
with open("app/places.json", "w") as fd:
    fd.write(output)

,lat,lon,name
0,51.895,-8.495,Gaol Cross
1,51.893,-8.505,Victoria Cross
2,51.889,-8.507,Dennehys Cross
3,53.333,-6.265,Kelly's Corner
4,53.333,-6.265,Coirnéal Uí Cheallaigh
...,...,...,...
22189,52.433,-9.401,Poulakilla
22190,52.430,-9.395,Poulaseer
22191,52.430,-9.402,Pouladewan
22192,52.440,-9.408,Poulavarnoge
